In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('playlists.csv')
df.head()  # muestra las primeras filas del DataFrame

df_copy = df.copy()

In [3]:
df.describe()  # proporciona estadísticas descriptivas para las columnas numéricas

,pid,num_tracks,num_albums,num_followers,modified_at,num_edits,duration_ms,num_artists
count,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1.000000e+06,1000000.000000,1.000000e+06,1000000.000000
mean,499999.500000,66.346428,49.597278,2.597746,1.476279e+09,17.655902,1.557968e+07,38.088211
std,288675.278932,53.669358,39.961064,128.851145,3.666992e+07,20.643254,1.285643e+07,30.282901
min,0.000000,5.000000,2.000000,1.000000,1.271376e+09,1.000000,9.753800e+04,3.000000
25%,249999.750000,26.000000,20.000000,1.000000,1.458950e+09,5.000000,5.995383e+06,16.000000
50%,499999.500000,49.000000,37.000000,1.000000,1.490227e+09,10.000000,1.142244e+07,29.000000
75%,749999.250000,92.000000,68.000000,1.000000,1.505434e+09,22.000000,2.143369e+07,52.000000
max,999999.000000,376.000000,244.000000,71643.000000,1.509494e+09,201.000000,6.350738e+08,238.000000


In [4]:
df.info()  # muestra información sobre el DataFrame, incluyendo los tipos de datos y la cantidad de valores no nulos

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 11 columns):
 #   Column         Non-Null Count    Dtype 
---  ------         --------------    ----- 
 0   pid            1000000 non-null  int64 
 1   name           999989 non-null   object
 2   description    18758 non-null    object
 3   num_tracks     1000000 non-null  int64 
 4   num_albums     1000000 non-null  int64 
 5   num_followers  1000000 non-null  int64 
 6   collaborative  1000000 non-null  bool  
 7   modified_at    1000000 non-null  int64 
 8   num_edits      1000000 non-null  int64 
 9   duration_ms    1000000 non-null  int64 
 10  num_artists    1000000 non-null  int64 
dtypes: bool(1), int64(8), object(2)
memory usage: 77.2+ MB


In [5]:
df.iloc[:,1:].describe()

,num_tracks,num_albums,num_followers,modified_at,num_edits,duration_ms,num_artists
count,1000000.000000,1000000.000000,1000000.000000,1.000000e+06,1000000.000000,1.000000e+06,1000000.000000
mean,66.346428,49.597278,2.597746,1.476279e+09,17.655902,1.557968e+07,38.088211
std,53.669358,39.961064,128.851145,3.666992e+07,20.643254,1.285643e+07,30.282901
min,5.000000,2.000000,1.000000,1.271376e+09,1.000000,9.753800e+04,3.000000
25%,26.000000,20.000000,1.000000,1.458950e+09,5.000000,5.995383e+06,16.000000
50%,49.000000,37.000000,1.000000,1.490227e+09,10.000000,1.142244e+07,29.000000
75%,92.000000,68.000000,1.000000,1.505434e+09,22.000000,2.143369e+07,52.000000
max,376.000000,244.000000,71643.000000,1.509494e+09,201.000000,6.350738e+08,238.000000
